In [4]:
import pytesseract
import cv2
import sys
import numpy as np

In [32]:
img = cv2.imread("../datasets/Manga109/Manga109_released_2021_12_30/images/AisazuNihaIrarenai/005.jpg")
kernel = np.ones((2, 2))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# thresh, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
# img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
# cv2.imshow("name", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
tess_config = "-l jpn_vert --psm 3"

h, w = img.shape
d = pytesseract.image_to_data(img, config=tess_config, output_type=pytesseract.Output.DICT)
n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("output/ocr-test.jpg", img)

True

In [5]:
img = cv2.imread("../datasets/Manga109/Manga109_released_2021_12_30/images/AisazuNihaIrarenai/005.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [11]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import easyocr


In [10]:
reader = easyocr.Reader(["ja"])
reader.readtext(img)

[([[346, 92], [370, 92], [370, 136], [346, 136]], 'き', 0.005661338057198151),
 ([[372, 92], [396, 92], [396, 116], [372, 116]], 'す', 0.9999947547981378),
 ([[392, 90], [446, 90], [446, 140], [392, 140]], '呼宮', 0.0035786419380895123),
 ([[394, 110], [418, 110], [418, 136], [394, 136]], 'び', 0.9997483650217909),
 ([[1094, 92], [1120, 92], [1120, 120], [1094, 120]], 'る', 0.9999988079074598),
 ([[1164, 90], [1196, 90], [1196, 142], [1164, 142]], '食', 0.8320708419018139),
 ([[127, 201], [145, 201], [145, 223], [127, 223]], 'テ', 0.9942628415174681),
 ([[144, 162], [168, 162], [168, 204], [144, 204]], 'あ', 0.102443296812055),
 ([[324, 92], [372, 92], [372, 232], [324, 232]], 'び', 0.0061993916832046025),
 ([[368, 126], [448, 126], [448, 214], [368, 214]], '開宮', 0.00397123886378513),
 ([[882, 64], [1052, 64], [1052, 232], [882, 232]], '慶三', 0.08047226228652755),
 ([[1078, 73], [1389, 73], [1389, 267], [1078, 267]],
  "'擬",
  0.04120480658915962),
 ([[1459, 89], [1497, 89], [1497, 175], [1459, 1